In [1]:
import pandas as pd
import numpy as np

In [17]:
keywords = pd.read_csv('keywords.csv')
sites = pd.read_csv('sites.csv')
rankings = pd.read_csv('rankings.csv')

rankings = rankings.merge(keywords, on='id_keyword')
rankings = rankings.merge(sites, on='id_site')
# rankings = rankings[['rank','keyword', 'domain', 'site']]
rankings

,id_ranking,rank,id_keyword,id_site,keyword,domain
0,66634,43,726,76,bachelor's degree in Economics,www.au.dk
1,200782,42,726,76,bachelor's degree in Economics,www.au.dk
2,156043,91,731,76,bachelor's degree in English Language,www.au.dk
3,313988,29,701,76,bachelor's degree in Archaeology,www.au.dk
4,402022,7,707,76,bachelor's degree in Business & Commercial Law,www.au.dk
5,874336,7,707,76,bachelor's degree in Business & Commercial Law,www.au.dk
6,1273764,7,707,76,bachelor's degree in Business & Commercial Law,www.au.dk
7,1406694,7,707,76,bachelor's degree in Business & Commercial Law,www.au.dk
8,1668060,7,707,76,bachelor's degree in Business & Commercial Law,www.au.dk
9,402400,79,753,76,bachelor's degree in German,www.au.dk


,id_ranking,rank,id_keyword,id_site,keyword
0,66634,43,726,76,bachelor's degree in Economics
1,67456,75,726,448,bachelor's degree in Economics
2,68484,62,726,966,bachelor's degree in Economics
3,75710,97,726,4344,bachelor's degree in Economics
4,77369,72,726,5150,bachelor's degree in Economics
5,78657,85,726,5791,bachelor's degree in Economics
6,79867,76,726,6410,bachelor's degree in Economics
7,79991,98,726,6472,bachelor's degree in Economics
8,80576,71,726,6748,bachelor's degree in Economics
9,80630,44,726,6777,bachelor's degree in Economics
